<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Get company followers
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Get_connections_from_network.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #linkedin #network #connections #naas_drivers #analytics #csv #html #image #content #plotly

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

This notebook get your LinkedIn profile connections and save it into a CSV file. 

## Input

### Get common variables, functions

In [1]:
# Except allow to run common.ipynb once using Naas Engine
try:
    %run "../common.ipynb"
except:
    %run "common.ipynb"

## Model

### Get followers

In [10]:
df_followers = get_data(LK_COMPANY_FOLLOWERS)
print('✅ Initial followers fetched:', len(df_followers))
df_followers.head(1)

✅ Initial followers fetched: 1902


,FIRSTNAME,LASTNAME,OCCUPATION,PROFILE_PICTURE,PROFILE_URL,PROFILE_ID,PUBLIC_ID,FOLLOWED_AT,DISTANCE,STATUS,DATE_EXTRACT
0,Justin,MAHORO,Technical Staff,NaN,https://www.linkedin.com/in/ACoAAA3XCo4B0Za6hb...,ACoAAA3XCo4B0Za6hbfdDuu7Lvlu1H50rjclgU4,justin-mahoro-15291965,2022-07-03 10:41:33,DISTANCE_3,Successfull,2022-07-03 13:24:23


### Update followers from company

In [11]:
def update_followers(df, key=None, min_updated_time=300):
    # Init output
    df_update = pd.DataFrame()
    
    # If df is empty then get all data
    if len(df) > 0:
        if "DATE_EXTRACT" in df.columns:
            last_update_date = df["DATE_EXTRACT"].max()
            time_last_update = datetime.now() - datetime.strptime(last_update_date, "%Y-%m-%d %H:%M:%S")
            minute_last_update = time_last_update.total_seconds() / 60
            if minute_last_update > min_updated_time:
                # If dataframe not empty, get last connections
                profiles = df[key].unique()
                start = 0
                count = 1
                while True:
                    tmp_new = linkedin.connect(LI_AT, JSESSIONID).company.get_followers(LINKEDIN_COMPANY_URL, start=start, count=count, limit=count)
                    # Check if existing profile in each call
                    if len(tmp_new) > 0:
                        tmp_new = tmp_new[~tmp_new.PROFILE_ID.isin(profiles)]
                        df_update = pd.concat([df_update, tmp_new])
                        if len(tmp_new) == 0:
                            break
                    # Get more profile
                    start += count
                print(f"-> New connections fetched: {len(df_update)}.")
            else:
                print(f"🛑 Nothing to update. Last update done {int(minute_last_update)} minutes ago.")
    else:
        df_update = linkedin.connect(LI_AT, JSESSIONID).company.get_followers(LINKEDIN_COMPANY_URL, count=100, limit=-1)
        
    # Concat and add extract date
    df = pd.concat([df_update, df]).drop_duplicates(key, keep="first")
    print(f"✅ {len(df)} followers fetched.")
    return df.reset_index(drop=True)
    
df_update = update_followers(df_followers,
                             key="PROFILE_ID")
df_update.head(1)

🛑 Nothing to update. Last update done 0 minutes ago.
✅ 1902 followers fetched.


,FIRSTNAME,LASTNAME,OCCUPATION,PROFILE_PICTURE,PROFILE_URL,PROFILE_ID,PUBLIC_ID,FOLLOWED_AT,DISTANCE,STATUS,DATE_EXTRACT
0,Justin,MAHORO,Technical Staff,NaN,https://www.linkedin.com/in/ACoAAA3XCo4B0Za6hb...,ACoAAA3XCo4B0Za6hbfdDuu7Lvlu1H50rjclgU4,justin-mahoro-15291965,2022-07-03 10:41:33,DISTANCE_3,Successfull,2022-07-03 13:24:23


## Output

### Save dataframe

In [12]:
save_data(df_update, LK_COMPANY_FOLLOWERS)

👌 Well done! Your Dependency has been sent to production. 

PS: to remove the "Dependency" feature, just replace .add by .delete
✅ Dataframe successfully saved in CSV: LinkedIn/Inputs/LINKEDIN_COMPANY_FOLLOWERS_70506391.csv
